In [1]:
import obspy
import matplotlib.pyplot as plt
import os
import datetime
import logging
import time

In [2]:
logger = logging.getLogger('Seismic data downloading')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('downloading.log')
# fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
ch = logging.StreamHandler()
# ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

In [3]:
center = (35.705, -117.504)
horizontal = 0.5
vertical = 0.5

In [4]:
starttime = obspy.UTCDateTime("2019-07-01")
endtime = obspy.UTCDateTime("2019-10-01")
# starttime = obspy.UTCDateTime("2019-04-01")
# endtime = obspy.UTCDateTime("2019-07-01")

In [5]:
from obspy.clients.fdsn import Client
client = Client("SCEDC")

In [ ]:
output_dir = "./mseed2"
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

In [64]:
events = client.get_events(starttime=starttime, 
                           endtime=endtime, 
                           minlatitude=center[0]-vertical,
                           maxlatitude=center[0]+vertical,
                           minlongitude=center[1]-horizontal,
                           maxlongitude=center[1]+horizontal,
                           minmagnitude=3, 
                           catalog="SCEDC", 
                           filename='events.xml')
# events = obspy.read_events('events.xml')
# events.plot('local', outfile="events.png")
# plt.savefig("events.png")
# logger.info("Number of events: {}".format(len(events)))

In [15]:
events = obspy.read_events('events.xml')
logger.info("Number of events: {}".format(len(events)))

Number of events: 984


In [ ]:
plt.figure(figsize=(15,15))
fig = plt.subplot()
# events.plot('local', outfile="events.png", fig=fig)
#fig = events.plot('local', fig=fig)
# plt.savefig("events.png")

In [63]:
stations = client.get_stations(network = "*",
                               station = "*",
                               starttime=starttime, 
                               endtime=endtime, 
                               minlatitude=center[0]-vertical,
                               maxlatitude=center[0]+vertical,
                               minlongitude=center[1]-horizontal,
                               maxlongitude=center[1]+horizontal,
                               level="response",
                               filename='stations.xml')
# stations = obspy.read_inventory('stations.xml')
# stations.plot('local', outfile="station.png")
# # plt.savefig("station.png")

In [65]:
stations = obspy.read_inventory('stations.xml')

In [ ]:
plt.figure(figsize=(15,15))
# fig = plt.gcf()
#stations.plot('local', outfile="station-event.png", size=100, fig=fig)
# plt.savefig("station.png")

In [ ]:
def download_all_stations(stations, starttime, endtime):
  max_retry = 5
  stream = obspy.Stream()
  logger.info("#########{}#########".format(starttime))
  for network in stations:
    for station in network:
      logger.info("********{}.{}********".format(network.code, station.code))
      retry = 0
      while retry < max_retry:
        try:
          tmp = client.get_waveforms(network.code, station.code, "*", "HN*", starttime, endtime)
          stream += tmp
          break
        except Exception as e:
#           logger.warning("Error {}.{}: {}".format(network.code, station.code,e))
          err = e
          retry += 1
          time.sleep(1)
          continue
      if retry == max_retry:
        logger.warning("MAX retries reach at {}.{} with error: {}".format(network.code, station.code, err))

  return stream

In [ ]:
tmp_start = starttime
one_day = datetime.timedelta(days=1)
while tmp_start < endtime:
  stream = download_all_stations(stations, tmp_start, tmp_start+one_day)
  stream.write(os.path.join(output_dir, "{}.mseed".format(tmp_start.datetime.strftime("%Y-%m-%d"))), format="MSEED")  
  tmp_start += one_day